In [60]:
import numpy as np

In [1]:
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
import tensorflow as tf
train_dataset_url = "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv"

train_dataset_fp = keras.utils.get_file(fname=os.path.basename(train_dataset_url),
                                           origin=train_dataset_url)

print("Local copy of the dataset file: {}".format(train_dataset_fp))
# column order in CSV file
column_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']

feature_names = column_names[:-1]
label_name = column_names[-1]

print("Features: {}".format(feature_names))
print("Label: {}".format(label_name))
class_names = ['Iris setosa', 'Iris versicolor', 'Iris virginica']
batch_size = 32

train_dataset = tf.data.experimental.make_csv_dataset(
    train_dataset_fp,
    batch_size,
    column_names=column_names,
    label_name=label_name,
    num_epochs=1)
features, labels = next(iter(train_dataset))

print(features)
plt.scatter(features['petal_length'],
            features['sepal_length'],
            c=labels,
            cmap='viridis')

plt.xlabel("Petal length")
plt.ylabel("Sepal length")
plt.show()


Using TensorFlow backend.


Local copy of the dataset file: /Users/ian/.keras/datasets/iris_training.csv
Features: ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
Label: species
Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.
OrderedDict([('sepal_length', <tf.Tensor: id=68, shape=(32,), dtype=float32, numpy=
array([4.7, 6.3, 7.4, 4.9, 6.9, 5.8, 5.1, 6.7, 6.7, 5.1, 6.4, 4.9, 4.9,
       5.8, 6.6, 4.8, 6.6, 4.9, 5. , 5. , 5. , 6. , 7.3, 6.8, 6.1, 5. ,
       5.8, 6.1, 5.6, 7.9, 6.4, 6. ], dtype=float32)>), ('sepal_width', <tf.Tensor: id=69, shape=(32,), dtype=float32, numpy=
array([3.2, 2.7, 2.8, 3. , 3.1, 2.7, 3.8, 3. , 3.1, 3.8, 2.8, 3.1, 3.1,
       2.7, 2.9, 3. , 3. , 3.1, 3.6, 3.3, 3.5, 2.2, 2.9, 3. , 2.8, 2.3,
       2.6, 2.9, 2.7, 3.8, 3.2, 3. ], dtype=float32)>), ('petal_length', <tf.Tensor: id=66, shape=(

<Figure size 640x480 with 1 Axes>

In [2]:
def pack_features_vector(features, labels):
  """Pack the features into a single array."""
  features = tf.stack(list(features.values()), axis=1)
  return features, labels
train_dataset = train_dataset.map(pack_features_vector)
features, labels = next(iter(train_dataset))

print(features[:5])

tf.Tensor(
[[5.4 3.  4.5 1.5]
 [6.  2.2 5.  1.5]
 [6.2 2.2 4.5 1.5]
 [4.8 3.1 1.6 0.2]
 [6.1 2.6 5.6 1.4]], shape=(5, 4), dtype=float32)


In [90]:
n_models = 4
models=[]
for _ in range(n_models):
    models.append(
        tf.keras.Sequential([
        tf.keras.layers.Dense(10, activation=tf.nn.relu, input_shape=(4,)),  # input shape required
        tf.keras.layers.Dense(10, activation=tf.nn.relu),
        tf.keras.layers.Dense(3)
])
    )
predictions = models[0](features)
predictions[:5]

<tf.Tensor: id=5043, shape=(5, 3), dtype=float32, numpy=
array([[ 3.4496427 , -0.7203656 ,  1.2507411 ],
       [ 3.542039  , -0.683318  ,  1.342955  ],
       [ 3.3160923 , -0.66011983,  1.276065  ],
       [ 2.3425536 , -0.92035985,  1.0660728 ],
       [ 4.010564  , -0.8338465 ,  1.5285674 ]], dtype=float32)>

In [4]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
def loss(model, x, y):
  y_ = model(x)

  return loss_object(y_true=y, y_pred=y_)


l = loss(models[0], features, labels)
print("Loss test: {}".format(l))

Loss test: 2.686091423034668


In [5]:
def grad(model, inputs, targets):
  with tf.GradientTape() as tape:
    loss_value = loss(model, inputs, targets)
  return loss_value, tape.gradient(loss_value, model.trainable_variables)

In [6]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)


In [7]:
# loss_value, grads = grad(model, features, labels)

# print("Step: {}, Initial Loss: {}".format(optimizer.iterations.numpy(),
#                                           loss_value.numpy()))

# optimizer.apply_gradients(zip(grads, model.trainable_variables))

# print("Step: {},         Loss: {}".format(optimizer.iterations.numpy(),
#                                           loss(model, features, labels).numpy()))

In [24]:
import re
from keras.models import Model


def insert_layer_nonseq(model, layer_regex, insert_layer_factory,
                        insert_layer_name=None, position='after'):

    # Auxiliary dictionary to describe the network graph
    network_dict = {'input_layers_of': {}, 'new_output_tensor_of': {}}

    # Set the input layers of each layer
    for layer in model.layers:
        for node in layer.outbound_nodes:
            layer_name = node.outbound_layer.name
            if layer_name not in network_dict['input_layers_of']:
                network_dict['input_layers_of'].update(
                        {layer_name: [layer.name]})
            else:
                network_dict['input_layers_of'][layer_name].append(layer.name)

    # Set the output tensor of the input layer
    network_dict['new_output_tensor_of'].update(
            {model.layers[0].name: model.input})

    # Iterate over all layers after the input
    for layer in model.layers[1:]:

        # Determine input tensors
        layer_input = [network_dict['new_output_tensor_of'][layer_aux] 
                for layer_aux in network_dict['input_layers_of'][layer.name]]
        if len(layer_input) == 1:
            layer_input = layer_input[0]

        # Insert layer if name matches the regular expression
        if re.match(layer_regex, layer.name):
            if position == 'replace':
                x = layer_input
            elif position == 'after':
                x = layer(layer_input)
            elif position == 'before':
                pass
            else:
                raise ValueError('position must be: before, after or replace')

            new_layer = insert_layer_factory
            if insert_layer_name:
                new_layer.name = insert_layer_name
            else:
                new_layer.name = '{}_{}'.format(layer.name, 
                                                new_layer.name)
            x = new_layer(x)
            print('Layer {} inserted after layer {}'.format(new_layer.name,
                                                            layer.name))
            if position == 'before':
                x = layer(x)
        else:
            x = layer(layer_input)

        # Set new output tensor (the original one, or the one of the inserted
        # layer)
        network_dict['new_output_tensor_of'].update({layer.name: x})

    return Model(inputs=model.inputs, outputs=x)


In [25]:
def test_insert_layer_nonseq():
    model1 = tf.keras.Sequential([
            tf.keras.layers.Dense(10, activation=tf.nn.relu, input_shape=(4,)),  # input shape required
            tf.keras.layers.Dense(10, activation=tf.nn.relu),
            tf.keras.layers.Dense(3)
    ])
    model2 = tf.keras.Sequential([
            tf.keras.layers.Dense(10, activation=tf.nn.relu, input_shape=(4,)),  # input shape required
            tf.keras.layers.Dense(10, activation=tf.nn.relu),
            tf.keras.layers.Dense(3)
    ])
    regex = model1.layers[0].outbound_nodes[0].output_tensors.name
    regex = model1.layers[1].outbound_nodes[0].inbound_layers.name
    insert_layer_naonseq(model1, regex, model2.layers[0],
                            insert_layer_name=None, position='replace')
    
test_insert_layer_nonseq()

AttributeError: Can't set the attribute "name", likely because it conflicts with an existing read-only @property of the object. Please choose a different name.

In [34]:
# tested in Keras 1.x
def copy_weights2model(w_source,target_model,certain_layer=""):
    l_nm=0
    for layer in target_model.layers:
        w_target=layer.get_weights()
        if len(w_target)>0:
            wk0=[w_source[2*l_nm],w_source[2*l_nm+1]]
            layer.set_weights(wk0)
            l_nm+=1
            if layer.name==certain_layer:
                break    

                # copy weights from one model to another model    
# tested in Keras 1.x    
def copyModel2Model(model_source,model_target, up_to_certain_layer=""):        
    for l_tg,l_sr in zip(model_target.layers, model_source.layers):
        wk0=l_sr.get_weights()
        l_tg.set_weights(wk0)
        if l_tg.name==up_to_certain_layer:
            print('break')
            break
    print("model source was copied into model target") 
copyModel2Model(models[0],models[1],"dense_33")

# def breed_models(model_source, model_target, certain_layer):
    


model source was copied into model target
model source was copied into model target


In [135]:
def model_clone(model, input_tensors):
    return tf.keras.models.clone_model(
        model,
        input_tensors=None,
        clone_function=None
    )

def breed_models(model_source1, model_source2, layer_list=list(), prob_select_2=list(), input_tensors=None):

    if not isinstance(prob_select_2, list) :
        prob_select_2 = [prob_select_2]*len(layer_list)
    model_target = model_clone(model_source1, input_tensors)
    for l_tg,l_sr, prob in zip(model_target.layers, model_source2.layers, prob_select_2):

        if l_tg.name in layer_list:
            roll = np.random.rand()
            if roll < prob:
                wk0 = l_sr.get_weights()
                l_tg.set_weights(wk0)

    return model_target

def test_breed_models():
    n_models = 2
    models=[]
    for _ in range(n_models):
        models.append(
                tf.keras.Sequential([
                tf.keras.layers.Dense(10, activation=tf.nn.relu, input_shape=(4,)),  # input shape required
                tf.keras.layers.Dense(10, activation=tf.nn.relu),
                tf.keras.layers.Dense(3)
            ])
        )
    model_new = breed_models(models[0],models[1], models[0].layers[0].name, 1)
    assert(np.all((np.squeeze(models[1].layers[0].get_weights()[0]-model_new.layers[0].get_weights()[0])) == 0))
    model_new = breed_models(models[0],models[1], models[1].layers[0].name, 0)
    assert(np.all((np.squeeze(models[0].layers[0].get_weights()[0]-model_new.layers[0].get_weights()[0])) != 0))
    
    
test_breed_models()

In [ ]:
def breed_conv_model_layer(model_source1, model_source2, layer_name,  prob_select_2=list(), 
                           input_tensors=None, 
                     ):
    if not isinstance(prob_select_2, list) :
        prob_select_2 = [prob_select_2]*len(layer_list)
    model_target = model_clone(model_source1, input_tensors)
    for l_tg,l_sr, prob in zip(model_target.layers, model_source2.layers, prob_select_2):
        if l_tg.name in name:
            roll = np.random.rand()
            if roll < prob:
                wk0 = l_sr.get_weights()
                l_tg.set_weights(wk0)
    
    
def test_breed_conv_models():
    n_models = 2
    models = []
    for _ in range(n_models):
        model =  tf.keras.Sequential()
        model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
        model.add(tf.keras.layers.MaxPooling2D((2, 2)))
        model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
        model.add(tf.keras.layers.MaxPooling2D((2, 2)))
        model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
        models.append(model)
        
    
        

In [170]:
model= tf.keras.Sequential()

model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(52, 52, 3)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model2= tf.keras.Sequential()
model2.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(52, 52, 3)))
model2.add(tf.keras.layers.MaxPooling2D((2, 2)))
model2.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model2.add(tf.keras.layers.MaxPooling2D((2, 2)))
model2.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))

In [189]:
# model.layers[0].weights[0].assign(model.layers[0].weights[0])
model2.layers[0].weights[0][:, :, :, 0]

<tf.Tensor: id=12542, shape=(3, 3, 3), dtype=float32, numpy=
array([[[-0.10087781,  0.0863834 , -0.01015307],
        [ 0.02318041, -0.06305347, -0.03850353],
        [-0.06927785,  0.12640204,  0.10435036]],

       [[ 0.02720787,  0.09027302,  0.06989743],
        [ 0.10577516, -0.10343836, -0.13326721],
        [ 0.07208455,  0.0445036 ,  0.09387802]],

       [[-0.06017017, -0.00910491, -0.0014572 ],
        [ 0.11111504,  0.01833007, -0.0326406 ],
        [-0.10435638, -0.02147277, -0.06814527]]], dtype=float32)>

In [238]:

def breed_convolution_layer_weights(model1_layer, model2_layer, prob_choose_2=list()):
    assert(model1_layer.weights[0].shape == model2_layer.weights[0].shape)
    assert(model1_layer.weights[1].shape == model2_layer.weights[1].shape)

    n_filters = model2_layer.weights[0].get_shape()[-1]
    
    if not isinstance(prob_choose_2, list):
        prob_list = []
        for _ in range(n_filters):
            prob_list.append(prob_choose_2)
    else:
        prob_list = prob_choose_2
    assert(n_filters == len(prob_list))
    output_list = []
    roll = 0

    output_biases = []
    for step_index, prob in enumerate(prob_list):
        if roll < prob:
            output_list.append(model2_layer.weights[0][:,:,:,step_index])
            output_biases.append(model2_layer.weights[1][step_index])            
        else:
            output_list.append(model1.layer.weights[0][:,:,:,step_index])
            output_biases.append(model1.layer.weights[1][step_index])

    return tf.stack(output_list), tf.stack(output_biases)

w, b = breed_convolution_layer_weights(model.layers[0], model2.layers[0], prob_choose_2=1)

In [247]:
w.shape

TensorShape([32, 3, 3, 3])

In [246]:
model2.layers[0].weights[0][0].shape

TensorShape([3, 3, 32])

In [ ]:
def clone_convolution_layer(model1_layer, model2_layer, model_tg_layer, prob=list()):


In [209]:
tensor_shape = model2.layers[0].weights[0].get_shape()
tensor_shape
output_list = []
prob = 1;
roll = 0
output_biases = []
for step_index in range(tensor_shape[-1]):
    if roll < prob:
        output_list.append(model2.layers[0].weights[0][:,:,:,step_index])
        output_biases.append(model2.layers[0].weights[1][step_index])

outputs = tf.stack(output_list)
output_biases = tf.stack(output_biases)

In [210]:
output_biases

<tf.Tensor: id=13553, shape=(32,), dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)>

In [211]:
model2.layers[0].weights[1]

<tf.Variable 'conv2d_21/bias:0' shape=(32,) dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)>

In [27]:
test_url = "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv"

test_fp = tf.keras.utils.get_file(fname=os.path.basename(test_url),
                                  origin=test_url)
test_dataset = tf.data.experimental.make_csv_dataset(
    test_fp,
    batch_size,
    column_names=column_names,
    label_name='species',
    num_epochs=1,
    shuffle=False)

test_dataset = test_dataset.map(pack_features_vector)

'dense_22'

In [33]:
## Note: Rerunning this cell uses the same model variables

# Keep results for plotting
train_loss_results = []
train_accuracy_results = []
test_accuracy_results = []
num_epochs = 201
test_accuracy = tf.keras.metrics.Accuracy()

for epoch in range(num_epochs):
    epoch_loss_avg = tf.keras.metrics.Mean()
    epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
    epoch_test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
    # Training loop - using batches of 32
    for (model_n, model) in enumerate(models):

        for x, y in train_dataset:
        # Optimize the model
            loss_value, grads = grad(model, x, y)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

            # Track progress
            epoch_loss_avg(loss_value)  # Add current batch loss
            # Compare predicted label to actual label
            epoch_accuracy(y, model(x))

        # End epoch
        train_loss_results.append(epoch_loss_avg.result())
        train_accuracy_results.append(epoch_accuracy.result())

        if epoch % 50 == 0:
            print("Model: {} Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(model_n, epoch,
                                                                epoch_loss_avg.result(),
                                                                epoch_accuracy.result()))

        for (x, y) in test_dataset:
            logits = model(x)
            prediction = tf.argmax(logits, axis=1, output_type=tf.int32)
            epoch_test_accuracy(y, model(x))

        if epoch % 50 == 0:
            print("Model: {} Epoch {:03d}: Accuracy: {:.3%}".format(model_n, epoch,
                                                                epoch_test_accuracy.result()))    

Model: 0 Epoch 000: Loss: 0.137, Accuracy: 96.667%
Model: 0 Epoch 000: Accuracy: 96.667%
Model: 1 Epoch 000: Loss: 0.130, Accuracy: 97.500%
Model: 1 Epoch 000: Accuracy: 96.667%
Model: 2 Epoch 000: Loss: 0.128, Accuracy: 97.778%
Model: 2 Epoch 000: Accuracy: 96.667%
Model: 3 Epoch 000: Loss: 0.124, Accuracy: 97.917%
Model: 3 Epoch 000: Accuracy: 95.833%
Model: 0 Epoch 050: Loss: 0.123, Accuracy: 98.333%
Model: 0 Epoch 050: Accuracy: 96.667%
Model: 1 Epoch 050: Loss: 0.118, Accuracy: 97.917%
Model: 1 Epoch 050: Accuracy: 96.667%
Model: 2 Epoch 050: Loss: 0.113, Accuracy: 98.056%
Model: 2 Epoch 050: Accuracy: 96.667%
Model: 3 Epoch 050: Loss: 0.110, Accuracy: 97.917%
Model: 3 Epoch 050: Accuracy: 95.833%
Model: 0 Epoch 100: Loss: 0.114, Accuracy: 97.500%
Model: 0 Epoch 100: Accuracy: 96.667%
Model: 1 Epoch 100: Loss: 0.114, Accuracy: 97.500%
Model: 1 Epoch 100: Accuracy: 96.667%
Model: 2 Epoch 100: Loss: 0.107, Accuracy: 97.778%
Model: 2 Epoch 100: Accuracy: 96.667%
Model: 3 Epoch 100: L

KeyboardInterrupt: 

In [ ]:
fig, axes = plt.subplots(2, sharex=True, figsize=(12, 8))
fig.suptitle('Training Metrics')

axes[0].set_ylabel("Loss", fontsize=14)
axes[0].plot(train_loss_results)

axes[1].set_ylabel("Accuracy", fontsize=14)
axes[1].set_xlabel("Epoch", fontsize=14)
axes[1].plot(train_accuracy_results)
plt.show()

In [11]:
test_url = "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv"

test_fp = tf.keras.utils.get_file(fname=os.path.basename(test_url),
                                  origin=test_url)

8192/573 [============================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 1us/step


In [12]:
test_dataset = tf.data.experimental.make_csv_dataset(
    test_fp,
    batch_size,
    column_names=column_names,
    label_name='species',
    num_epochs=1,
    shuffle=False)

test_dataset = test_dataset.map(pack_features_vector)

In [13]:
test_accuracy = tf.keras.metrics.Accuracy()

for (x, y) in test_dataset:
  logits = model(x)
  prediction = tf.argmax(logits, axis=1, output_type=tf.int32)
  test_accuracy(prediction, y)

print("Test set accuracy: {:.3%}".format(test_accuracy.result()))

Test set accuracy: 96.667%


In [16]:
predict_dataset = tf.convert_to_tensor([
    [5.1, 3.3, 1.7, 0.5,],
    [5.9, 3.0, 4.2, 1.5,],
    [6.9, 3.1, 5.4, 2.1]
])

predictions = model(predict_dataset)

for i, logits in enumerate(predictions):
  class_idx = tf.argmax(logits).numpy()
  p = tf.nn.softmax(logits)[class_idx]
  name = class_names[class_idx]
  print("Example {} prediction: {} ({:4.1f}%)".format(i, name, 100*p))

Example 0 prediction: Iris setosa (97.4%)
Example 1 prediction: Iris versicolor (91.7%)
Example 2 prediction: Iris virginica (75.0%)
